In [2]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver
from tqdm.notebook import tqdm
import time

In [25]:
# 데이터 수집할 키워드 지정
keyword = input()
keyword

디즈니


'디즈니'

In [32]:
# 크롬 실행
driver = webdriver.Chrome("C:\chrome\chromedriver.exe") # 크롬드라이버가 위치한 경로설정
driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}".format(keyword))
time.sleep(3)

In [19]:
# 뉴스 url 수집하기
things = driver.find_elements_by_link_text('네이버뉴스')  

# url 주소를 저장할 리스트 생성
url_list = []

for thing in things:
    url = thing.get_attribute('href')
    url_list.append(url)

# url 주소 수집결과 확인
print(len(url_list))
url_list

34


['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=293&aid=0000034734',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=277&aid=0004904627',
 'https://sports.news.naver.com/news.nhn?oid=442&aid=0000134209',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0002956287',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=009&aid=0004795687',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=609&aid=0000442404',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=144&aid=0000737424',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=057&aid=0001577395',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012399847',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=374&aid=0000244631',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002222324',
 'https://news.naver.com/main/read

In [20]:
# 크롬 닫기
driver.close()

In [21]:
df = pd.DataFrame({"url":url_list})
df.to_csv('./dataset/navernews_urls.csv')

# 댓글 수집하기

In [3]:
# 저장해둔 url 불러오기
df = pd.read_csv('./dataset/navernews_urls.csv')

In [13]:
# 뉴스 크롬창 띄우기
driver = webdriver.Chrome("C:\chrome\chromedriver.exe")
driver.get(df['url'][0])

In [7]:
# 기사 댓글 갯수
review_count = driver.find_element_by_css_selector('.u_cbox_count').text
review_count = int(review_count.replace(',',''))
review_count

5

In [27]:
# 더보기 클릭 횟수
moreview_num = review_count//20
moreview_num

0

In [6]:
# 댓글 버튼 클릭
driver.find_element_by_css_selector('.lo_txt').click()
time.sleep(1)

In [ ]:
k=0
while k <=moreview_num: # 더보기 횟수만큼 반복
    try:
        driver.find_element_by_css_selector('.u_cbox_page_more').click() # 더보기 버튼 클릭
        time.sleep(1)
        k=k+1
    except:
        break # 에러나면 클릭 반복문을 빠져나가라

In [10]:
# review 수집하기
review_list = []

overlays1 = ".u_cbox_text_wrap"
reviews = driver.find_elements_by_css_selector(overlays1)

for review in tqdm(reviews):    
    review = review.text
    review_list.append(review)

print(len(review_list))
review_list


5


['아갈만 털지말고 좀 출시일을 내라고 매번 지겨운 예정예정',
 '디즈니 주가좀 어떻게.. 250달러 언제가나여',
 '8월초에나오면좋겠네요 마블퓨처레볼루션',
 '기사야 빵빵 터져라!!!',
 '디즈니 매직킹덤... 예전엔 재밌었는데 이젠 현질을 너무 해야돼서 때려침...']

# 기사 내용 수집하기

In [14]:
# 기사 제목
title = driver.find_element_by_css_selector('.tts_head')
title = title.text
print(title)

[오~컬쳐]디즈니, 한국서 게임 영토 확장…'디즈니+'로 시너지 낼까


In [16]:
# 기사 날짜
date = driver.find_element_by_css_selector('.t11')
date = date.text
print(date)

2021.05.18. 오전 10:42


In [17]:
# 기사 추천수 날짜
up = driver.find_element_by_css_selector('.u_cnt._count')
up = up.text
print(up)

7


# 기사 감정 수집

In [20]:
# 좋아요
like = driver.find_element_by_css_selector(".end_btn .u_likeit_list.good .u_likeit_list_count._count")
like = like.text
print(like)

#훈훈해요
warm = driver.find_element_by_css_selector('.end_btn .u_likeit_list.warm .u_likeit_list_count._count')
warm = warm.text
print(warm)

# 슬퍼요
sad = driver.find_element_by_css_selector('.end_btn .u_likeit_list.sad .u_likeit_list_count._count')
sad = sad.text
print(sad)

# 화나요
angry = driver.find_element_by_css_selector('.end_btn .u_likeit_list.angry .u_likeit_list_count._count')
angry = angry.text
print(angry)

# 원해요(후속기사)
want = driver.find_element_by_css_selector('.end_btn .u_likeit_list.want .u_likeit_list_count._count')
want = want.text
print(want)

5
0
0
0
5


In [21]:
target_info = {}

target_info['기사명'] = title
target_info['날짜'] = date
target_info['추천수'] = up
target_info['댓글'] = review_list
target_info['좋아요'] = like
target_info['훈훈해요']= warm
target_info['슬퍼요']= sad
target_info['화나요']= angry
target_info['후속기사_원해요']= want

target_info

{'기사명': "[오~컬쳐]디즈니, 한국서 게임 영토 확장…'디즈니+'로 시너지 낼까",
 '날짜': '2021.05.18. 오전 10:42',
 '추천수': '7',
 '댓글': ['아갈만 털지말고 좀 출시일을 내라고 매번 지겨운 예정예정',
  '디즈니 주가좀 어떻게.. 250달러 언제가나여',
  '8월초에나오면좋겠네요 마블퓨처레볼루션',
  '기사야 빵빵 터져라!!!',
  '디즈니 매직킹덤... 예전엔 재밌었는데 이젠 현질을 너무 해야돼서 때려침...'],
 '좋아요': '5',
 '훈훈해요': '0',
 '슬퍼요': '0',
 '화나요': '0',
 '후속기사_원해요': '5'}

# 통합 코드

In [23]:
dict = {}

for i in tqdm(range(0, len(df['url']))):
    try:
        # 뉴스 크롬창 띄우기
        driver = webdriver.Chrome("C:\chrome\chromedriver.exe")
        driver.get(df['url'][i])
        time.sleep(1)
        
        # 기사 데이터 수집
        title = driver.find_element_by_css_selector('.tts_head').text
        date = driver.find_element_by_css_selector('.t11').text
        up = driver.find_element_by_css_selector('.u_cnt._count').text
        like = driver.find_element_by_css_selector(".end_btn .u_likeit_list.good .u_likeit_list_count._count").text
        warm = driver.find_element_by_css_selector('.end_btn .u_likeit_list.warm .u_likeit_list_count._count').text
        sad = driver.find_element_by_css_selector('.end_btn .u_likeit_list.sad .u_likeit_list_count._count').text
        angry = driver.find_element_by_css_selector('.end_btn .u_likeit_list.angry .u_likeit_list_count._count').text
        want = driver.find_element_by_css_selector('.end_btn .u_likeit_list.want .u_likeit_list_count._count').text
        
        # 기사 댓글 갯수
        review_count = int(driver.find_element_by_css_selector('.u_cbox_count').text.replace(',', ''))

        # 더보기 클릭
        moreview_num = review_count//20

        # 댓글 버튼 클릭
        driver.find_element_by_css_selector('.lo_txt').click()
        time.sleep(1)

        # 더보기 여러번 클릭
        k=0
        while k <=moreview_num: # 더보기 횟수만큼 반복
            try:
                driver.find_element_by_css_selector('.u_cbox_page_more').click() # 더보기 버튼 클릭
                time.sleep(1)
                k=k+1
            except:
                break # 에러나면 클릭 반복문을 빠져나가라   
                
        # review 수집하기
        review_list = []

        overlays1 = ".u_cbox_text_wrap"
        reviews = driver.find_elements_by_css_selector(overlays1)

        for review in tqdm(reviews):    
            review = review.text
            review_list.append(review)  
            
        target_info = {}

        target_info['기사명'] = title
        target_info['날짜'] = date
        target_info['추천수'] = up
        target_info['댓글'] = review_list
        target_info['좋아요'] = like
        target_info['훈훈해요']= warm
        target_info['슬퍼요']= sad
        target_info['화나요']= angry
        target_info['후속기사_원해요']= want
        
        dict[i]=target_info
        print(title, '( 댓글 수:', len(review_list),')')
        
        driver.close()
        time.sleep(1)
        
    except:
        driver.close()
        continue
        
print(len(dict))
dict


[오~컬쳐]디즈니, 한국서 게임 영토 확장…'디즈니+'로 시너지 낼까 (댓글 수: 5 )



디즈니코리아, 韓게임사 협업 지속…니모·오메가 플라이트 출격 (댓글 수: 1 )



워너-디스커버리, 49조원대 합병…"넷플릭스·디즈니 추격"(종합) (댓글 수: 0 )



“넷플릭스·디즈니+ 잡는다”…AT&T, 디스커버리와 합병 계약 [장가희 기자의 뉴스픽] (댓글 수: 7 )



美 워너-디스커버리 합병…디즈니·넷플릭스 추격 (댓글 수: 2 )


...


달라진 서학개미 장바구니… 보잉·디즈니 경기회복株 늘렸다 (댓글 수: 3 )



'코로나 특수 끝?' 넷플릭스 쫓던 디즈니+도 가입자 증가세↓ (댓글 수: 4 )



디즈니+, 2분기 신규가입 870만명 그쳐…시장 기대치 하회 (댓글 수: 1 )



OTT 성장 둔화하나…디즈니+ 가입 증가 시장 기대 못 미쳐 (댓글 수: 0 )

27


{0: {'기사명': "[오~컬쳐]디즈니, 한국서 게임 영토 확장…'디즈니+'로 시너지 낼까",
  '날짜': '2021.05.18. 오전 10:42',
  '추천수': '7',
  '댓글': ['아갈만 털지말고 좀 출시일을 내라고 매번 지겨운 예정예정',
   '디즈니 주가좀 어떻게.. 250달러 언제가나여',
   '8월초에나오면좋겠네요 마블퓨처레볼루션',
   '기사야 빵빵 터져라!!!',
   '디즈니 매직킹덤... 예전엔 재밌었는데 이젠 현질을 너무 해야돼서 때려침...'],
  '좋아요': '5',
  '훈훈해요': '0',
  '슬퍼요': '0',
  '화나요': '0',
  '후속기사_원해요': '5'},
 1: {'기사명': '디즈니코리아, 韓게임사 협업 지속…니모·오메가 플라이트 출격',
  '날짜': '2021.05.18. 오전 9:39',
  '추천수': '',
  '댓글': ['누가 디즈니 주가좀 올려주세요. 사망직전.'],
  '좋아요': '1',
  '훈훈해요': '0',
  '슬퍼요': '0',
  '화나요': '0',
  '후속기사_원해요': '0'},
 8: {'기사명': '워너-디스커버리, 49조원대 합병…"넷플릭스·디즈니 추격"(종합)',
  '날짜': '2021.05.18. 오전 12:04',
  '추천수': '',
  '댓글': [],
  '좋아요': '3',
  '훈훈해요': '0',
  '슬퍼요': '0',
  '화나요': '0',
  '후속기사_원해요': '0'},
 9: {'기사명': '“넷플릭스·디즈니+ 잡는다”…AT&T, 디스커버리와 합병 계약 [장가희 기자의 뉴스픽]',
  '날짜': '2021.05.18. 오전 6:30',
  '추천수': '',
  '댓글': ['아마존을 뺐내.Ott시장 2위가 아마존 3위가 디즈니+인데 왜 Ott시장을 넷플릭스와 디즈니+가 독점하다시피 한다고 기사를 쓴건지.기자분은 기사쓸 때 팩트체크란 것도 안하나?',
   '넷플릭스 볼것없다~앞으로 주가 떨어짐 디즈니에 투자하라',
 

In [26]:
import pandas as pd
result_df = pd.DataFrame.from_dict(dict,'index')

result_df.to_excel('./dataset/naver_article_({}).xlsx'.format(keyword))

In [27]:
result_df

,기사명,날짜,추천수,댓글,좋아요,훈훈해요,슬퍼요,화나요,후속기사_원해요
0,"[오~컬쳐]디즈니, 한국서 게임 영토 확장…'디즈니+'로 시너지 낼까",2021.05.18. 오전 10:42,7,"[아갈만 털지말고 좀 출시일을 내라고 매번 지겨운 예정예정, 디즈니 주가좀 어떻게....",5,0,0,0,5
1,"디즈니코리아, 韓게임사 협업 지속…니모·오메가 플라이트 출격",2021.05.18. 오전 9:39,,[누가 디즈니 주가좀 올려주세요. 사망직전.],1,0,0,0,0
8,"워너-디스커버리, 49조원대 합병…""넷플릭스·디즈니 추격""(종합)",2021.05.18. 오전 12:04,,[],3,0,0,0,0
9,"“넷플릭스·디즈니+ 잡는다”…AT&T, 디스커버리와 합병 계약 [장가희 기자의 뉴스픽]",2021.05.18. 오전 6:30,,[아마존을 뺐내.Ott시장 2위가 아마존 3위가 디즈니+인데 왜 Ott시장을 넷플릭...,2,0,0,6,0
10,美 워너-디스커버리 합병…디즈니·넷플릭스 추격,2021.05.18. 오전 7:42,1,"[디즈니가 최고다, 3등은 살아남기 힘들다. 2등도 괴로운데.]",4,0,0,0,2
11,"워너미디어-디스커버리, 49조원 합병…넷플릭스‧디즈니+ 쫓는다",2021.05.18. 오전 11:16,1,[],2,0,0,0,0
12,"AT&T, 디스커버리 합병 추진…넷플·디즈니 아성에 도전(종합)",2021.05.17. 오전 11:13,,[합쳐도 모자른판에 더 먹겄다고 따로 하는 신세들이 무슨 보호임],4,0,0,0,0
13,"AT&T, '다큐 왕국' 디스커버리 품었다",2021.05.17. 오후 5:23,1,[작성자에 의해 삭제된 댓글입니다.],2,0,0,1,2
14,"AT&T-디스커버리 합병…""넷플릭스·디즈니 추격""",2021.05.17. 오후 10:36,,[],0,0,0,0,0
15,"AT&T-디스커버리 합병…""넷플릭스·디즈니 추격""",2021.05.17. 오후 8:31,,[],0,0,0,0,0
